# Data Wrangling for Taiwan Credit Analysis
This notebook details the data wrangling done to prepare this dataset to be used to create a classification algorithm that will be used to predict whether someone will default the next month, based on the past six months of credit card payments and personal information. 

In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from pandas.io.json import json_normalize
import seaborn

In [3]:
# Load up the dataset
defaults_df = pd.DataFrame.from_csv('default_of_credit_card_clients.csv')
print(defaults_df.shape)
defaults_df.head()

(30001, 24)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


We can see that the first row is actually the column IDs. Next step will be to move that up so that those are the column names.

In [4]:
# Getting rid of first line mistake - 
new_header = defaults_df.iloc[0]
defaults_df = defaults_df[1:]
defaults_df.rename(columns = new_header)

#Renaming columns
new_header = list(new_header)
new_header[-1] = 'Y'
defaults_df.columns = new_header

print(defaults_df.shape)
defaults_df.head()

(30000, 24)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


I want to see if there are any missing columns, if all of the datatypes are numeric, and if so, do something about those two issues.

In [5]:
# Data types
defaults_df.dtypes

LIMIT_BAL    object
SEX          object
EDUCATION    object
MARRIAGE     object
AGE          object
PAY_0        object
PAY_2        object
PAY_3        object
PAY_4        object
PAY_5        object
PAY_6        object
BILL_AMT1    object
BILL_AMT2    object
BILL_AMT3    object
BILL_AMT4    object
BILL_AMT5    object
BILL_AMT6    object
PAY_AMT1     object
PAY_AMT2     object
PAY_AMT3     object
PAY_AMT4     object
PAY_AMT5     object
PAY_AMT6     object
Y            object
dtype: object

Looks like I'll need to convert all of the values to numeric. I will do that later, but at least I know now.

In [6]:
# Missing/Null values?
defaults_df.isnull().sum()

LIMIT_BAL    0
SEX          0
EDUCATION    0
MARRIAGE     0
AGE          0
PAY_0        0
PAY_2        0
PAY_3        0
PAY_4        0
PAY_5        0
PAY_6        0
BILL_AMT1    0
BILL_AMT2    0
BILL_AMT3    0
BILL_AMT4    0
BILL_AMT5    0
BILL_AMT6    0
PAY_AMT1     0
PAY_AMT2     0
PAY_AMT3     0
PAY_AMT4     0
PAY_AMT5     0
PAY_AMT6     0
Y            0
dtype: int64

Zero null values. No need to worry about imputing or filling in missing data.

## One-hot encoding
One-hot encoding is useful in python to train any type of model that contains categorical features. In this case, there are 9 different features that contain categorical values. Within those columns, there are varying options for distinction, such as sex having 2 options and history of past payment having 10 different values. One-hot encoding will split each of these columns into columns that have one category in the column, and whether or not that value is present in the row. Going back to the 'sex' category, this will split it into two columns called: "SEX_male" and "SEX_female", each of which will only contain binary variables.

To do this, I will use pandas to mutate the dataframe into something that is ready to use for ML algorithms.

In [7]:
clf_df = pd.get_dummies(defaults_df, columns=["SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"],
               prefix=["SEX", "EDU", "MARRY", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"])
clf_df.shape

(30000, 92)

Creted about 80 new columns that are permutations of all the unique values in the categorical columns.

In [8]:
# Finally, converting all values to numeric
clf_df = clf_df.apply(pd.to_numeric)

In [9]:
clf_df.dtypes

LIMIT_BAL    int64
AGE          int64
BILL_AMT1    int64
BILL_AMT2    int64
BILL_AMT3    int64
BILL_AMT4    int64
BILL_AMT5    int64
BILL_AMT6    int64
PAY_AMT1     int64
PAY_AMT2     int64
PAY_AMT3     int64
PAY_AMT4     int64
PAY_AMT5     int64
PAY_AMT6     int64
Y            int64
SEX_1        uint8
SEX_2        uint8
EDU_0        uint8
EDU_1        uint8
EDU_2        uint8
EDU_3        uint8
EDU_4        uint8
EDU_5        uint8
EDU_6        uint8
MARRY_0      uint8
MARRY_1      uint8
MARRY_2      uint8
MARRY_3      uint8
PAY_0_-1     uint8
PAY_0_-2     uint8
             ...  
PAY_4_-2     uint8
PAY_4_0      uint8
PAY_4_1      uint8
PAY_4_2      uint8
PAY_4_3      uint8
PAY_4_4      uint8
PAY_4_5      uint8
PAY_4_6      uint8
PAY_4_7      uint8
PAY_4_8      uint8
PAY_5_-1     uint8
PAY_5_-2     uint8
PAY_5_0      uint8
PAY_5_2      uint8
PAY_5_3      uint8
PAY_5_4      uint8
PAY_5_5      uint8
PAY_5_6      uint8
PAY_5_7      uint8
PAY_5_8      uint8
PAY_6_-1     uint8
PAY_6_-2    